# SQL generation test

In [2]:
from google.cloud import bigquery
from sqlalchemy import *
from sqlalchemy.engine import create_engine
from sqlalchemy.schema import *
import os
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.llms.openai import OpenAI
from langchain.agents import AgentExecutor
service_account_file = "" # Change to where your service account key file is located
project = "recruit-notice"
dataset = "test"
table = "content"
sqlalchemy_url = f'bigquery://{project}/{dataset}?credentials_path={service_account_file}'

In [6]:
from langchain.chat_models import ChatOpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = ""
# KEY는 디코로 문의 주세요!

In [10]:
db = SQLDatabase.from_uri(sqlalchemy_url)
llm = ChatOpenAI(temperature=0.5)
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
llm=llm,
toolkit=toolkit,
verbose=True,
top_k=10,
)

/opt/homebrew/Caskroom/miniforge/base/envs/dbt/lib/python3.10/site-packages/google/cloud/bigquery/client.py:572: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


In [14]:
agent_executor.run("Find the url in content table that contain 'QA 엔지니어'")



> Entering new AgentExecutor chain...
I need to find the url in the content table that contains the text 'QA 엔지니어'. I should use a SQL query to search for this information.
Action: sql_db_query
Action Input: SELECT url FROM content WHERE content LIKE '%QA 엔지니어%'Error: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 400 POST https://bigquery.googleapis.com/bigquery/v2/projects/recruit-notice/queries?prettyPrint=false: No matching signature for operator LIKE for argument types: STRUCT<url STRING, title STRING, job_name STRING, ...>, STRING. Supported signatures: STRING LIKE STRING; BYTES LIKE BYTES at [1:31]
[SQL: SELECT url FROM content WHERE content LIKE '%%QA 엔지니어%%']
(Background on this error at: https://sqlalche.me/e/14/4xp6)The query I used is incorrect. I need to check the schema of the content table to see the correct field name to use in the query.
Action: sql_db_schema
Action Input: content
CREATE TABLE `content` (
	`url` STRING, 
	`title` STRING, 
	`job_name` STRING, 

"The url in the content table that contains the text 'QA 엔지니어' is retrieved using the SQL query: SELECT url FROM content WHERE description LIKE '%QA 엔지니어%'"

# RAG Test

In [35]:
# import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import BigQueryLoader
from langchain_openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [25]:
BASE_QUERY = """
SELECT
  *
FROM `recruit-notice.test.content`
LIMIT 1000
"""

In [33]:
loader = BigQueryLoader(BASE_QUERY, project="recruit-notice")
data = loader.load()

/opt/homebrew/Caskroom/miniforge/base/envs/dbt/lib/python3.10/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [34]:
print(data)

[Document(page_content='url: https://www.wanted.co.kr/wd/105446\ntitle: Software Engineer, Backend\njob_name: 루비온레일즈 개발자\ncompany_name: 버킷플레이스(오늘의집)\ntech_list: []\ndeadline: None\nlocation: None\nmain_work: [\'애플리케이션 컴포넌트 설계, 개발, 테스트 및 운영\', \'MSA를 위한 서비스 Re-Architecting\', \'서비스 운영을 위한 관리자 페이지 개발 및 개선\', \'서비스 아키텍처 설계부터 출시까지 전 과정 참여\', \'Spring Framework + Kotlin, Ruby on Rails를 이용하여 서비스 개발 및 운영\']\nacademic_background: None\ncareer: None\npreferences: [\'소규모 프로젝트 리딩 경험이 있는 분\', \'대용량 트래픽 처리에 대한 이해 및 서비스 경험이 있는 분\', \'전체 SDLC(소프트웨어 생명주기)에 대한 경험이 있는 분\', \'애자일 스프린트, MVP 단위 개발 프로세스에서의 개발 경험이 있는 분\', \'AWS와 같은 Public Cloud 작업 경험이 있는 분\', \'인테리어 산업에 대한 높은 관심과 이해도가 있는 분\']\nwelfare: [\'몰입과 성장을 위한 근무환경- 근무시간 자율 (6시~22시)- 최대 주 3회 재택근무제 운영- 결재 필요 없는 자유로운 휴가 사용- 도서, 컨퍼런스, 교육 지원- 조/중/석식 지원 - 사내카페 무료 운영 - 허먼밀러 의자 지급\', \'인재에 대한 아낌없는 보상- 최고의 인재에게 최고의 보상- 입사자 전원 스톡옵션 부여- 고기여자 대상의 스톡옵션 부여- 성과에 따른 전사 인센티브 지급\', \'함께 오래 가기 위한 케어- 건강검진 지원 (본인/배우자/양가 부모님)- 단체보험 지원 (본인/배우자/자녀)- 리프레시 휴가 및 장기근속 포상- 마음케어 

In [36]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

In [37]:
splits = text_splitter.split_documents(data)

In [39]:
vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())

In [40]:
retriever = vectorstore.as_retriever()

In [42]:
prompt = hub.pull("rlm/rag-prompt")

In [48]:
llm = ChatOpenAI(temperature=0.1)

In [43]:
def format_docs(docs):
    # 검색한 문서 결과를 하나의 문단으로 합쳐줍니다.
    return "\n\n".join(doc.page_content for doc in docs)

In [49]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [50]:
question = "Python 개발자 모집공고의 url을 알려줘."
response = rag_chain.invoke(question)

# 결과 출력
print(f"[HUMAN]\n{question}\n")
print(f"[AI]\n{response}")

[HUMAN]
Python 개발자 모집공고의 url을 알려줘.

[AI]
Python 개발자 모집공고의 url은 https://www.jumpit.co.kr/position/17003, https://www.jumpit.co.kr/position/5662, https://www.jumpit.co.kr/position/20472 입니다.
